In [1]:
import requests
import numpy as np
import pandas as pd

from sklearn import preprocessing

import time

from tqdm import tqdm
from tqdm import trange

import copy

from multiprocessing import Pool # Pool import하기

In [2]:
import os

DATA_DIR = ""
DUMP_FILE = os.path.join(DATA_DIR, "정규화후.pkl")

def dump_dataframes(dataframes):
    pd.to_pickle(dataframes, DUMP_FILE)

def load_dataframes():
    return pd.read_pickle(DUMP_FILE)
# dump_dataframes(match_fin)

In [3]:
user_mastery = load_dataframes()

In [109]:
user_mastery

,266,103,84,12,32,34,1,523,22,136,...,157,777,83,350,154,238,115,26,142,143
vvi2,0.095098,0.003870,0.010181,0.038616,0.008656,0.000000,0.000000,0.003089,0.040411,0.0,...,0.042128,0.000000,0.000000,0.000000,0.116988,0.015208,0.000000,0.000000,0.011005,0.000000
liu de hua,0.005644,0.002093,0.008588,0.000000,0.000000,0.000000,0.000000,0.000602,0.001505,0.0,...,0.006883,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001250,0.000000
DWG Canyon,0.600000,0.005527,0.032101,0.005838,0.000000,0.000000,0.000000,0.008133,0.002785,0.0,...,0.000537,0.000000,0.003380,0.000000,0.036771,0.000000,0.000000,0.037176,0.059218,0.000642
can see you,0.002120,0.000000,0.000000,0.000000,0.000000,0.000000,0.000182,0.000000,0.001260,0.0,...,0.000304,0.001033,0.000000,0.002516,0.001648,0.000000,0.000971,0.000000,0.000000,0.000000
T1 Burdol,0.331610,0.000000,0.314465,0.011344,0.000000,0.000000,0.000000,0.001831,0.000000,0.0,...,0.028427,0.000000,0.033434,0.055608,0.000000,0.002112,0.000000,0.000000,0.000456,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
사 용 수,0.270882,0.000000,0.026930,0.000000,0.000000,0.000000,0.000000,0.000000,0.006594,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
카게게,0.273359,0.000000,0.332258,0.016999,0.000000,0.000000,0.000000,0.063358,0.017517,0.0,...,0.174470,0.040547,0.000000,0.027708,0.000000,0.241729,0.035372,0.002070,0.266571,0.000000
IAKIL,0.009418,0.000598,0.150717,0.000000,0.000000,0.000739,0.009779,0.614114,0.272830,0.0,...,0.310345,0.037492,0.000000,0.064308,0.005267,0.025181,0.025282,0.005606,0.207947,0.006305
jooI,0.095098,0.003870,0.010181,0.038616,0.008656,0.000000,0.000000,0.003089,0.040411,0.0,...,0.042128,0.000000,0.000000,0.000000,0.116988,0.015208,0.000000,0.000000,0.011005,0.000000


In [66]:
user_mastery.index

Index(['vvi2', 'liu de hua', 'DWG Canyon', 'can see you', 'T1 Burdol',
       '버기해적단 버기', 'ehfdkdhsakql', 'XiaoHuaSheng7', 'SANDBOX Howling',
       'Tar2an',
       ...
       'DRX 홍창현', 'KronGG', 'dongshiyuan1hao', 'Axiyo', '준탱이528', '사 용 수',
       '카게게', 'IAKIL', 'jooI', 'Bmav'],
      dtype='object', length=300)

## 유저-유저 유사도 행렬 구하기

[딥 러닝을 이용한 자연어 처리 입문](https://wikidocs.net/24603)

cosine similarity 사용

### 데이터 추가했을 때 u*u 만큼 하지말고 +1 padding하는 느낌으로 연산해야 될 듯?

In [27]:
from numpy import dot
from numpy.linalg import norm

def cosine_sim(A,B):
    return dot(A, B)/(norm(A)*norm(B))

In [29]:
user_mastery.loc['vvi2',:]

266    0.095098
103    0.003870
84     0.010181
12     0.038616
32     0.008656
         ...   
238    0.015208
115    0.000000
26     0.000000
142    0.011005
143    0.000000
Name: vvi2, Length: 150, dtype: float64

In [28]:
cosine_sim(user_mastery.loc['vvi2',:].values,user_mastery.loc['vvi2',:].values)

1.0000000000000002

In [36]:
l = len(user_mastery.index)

In [68]:
user_user_sim =np.mat(np.zeros((l, l)))

In [69]:
for i in range(l):
    for j in range(l):
        user_user_sim[i,j] = cosine_sim(user_mastery.iloc[i,:],user_mastery.iloc[j,:])

In [70]:
user_user_sim

matrix([[1.        , 0.02245791, 0.64739604, ..., 0.17705798, 1.        ,
         0.02245791],
        [0.02245791, 1.        , 0.02477776, ..., 0.015395  , 0.02245791,
         1.        ],
        [0.64739604, 0.02477776, 1.        , ..., 0.14406472, 0.64739604,
         0.02477776],
        ...,
        [0.17705798, 0.015395  , 0.14406472, ..., 1.        , 0.17705798,
         0.015395  ],
        [1.        , 0.02245791, 0.64739604, ..., 0.17705798, 1.        ,
         0.02245791],
        [0.02245791, 1.        , 0.02477776, ..., 0.015395  , 0.02245791,
         1.        ]])

In [72]:
df_user_user_sim = pd.DataFrame(data=user_user_sim,index=user_mastery.index, columns = user_mastery.index)

In [73]:
df_user_user_sim

,vvi2,liu de hua,DWG Canyon,can see you,T1 Burdol,버기해적단 버기,ehfdkdhsakql,XiaoHuaSheng7,SANDBOX Howling,Tar2an,...,DRX 홍창현,KronGG,dongshiyuan1hao,Axiyo,준탱이528,사 용 수,카게게,IAKIL,jooI,Bmav
vvi2,1.000000,0.022458,0.647396,0.071693,0.190528,0.710296,0.038351,0.820694,0.172132,0.695309,...,0.025006,0.156684,0.527983,0.219524,0.040231,0.161267,0.102552,0.177058,1.000000,0.022458
liu de hua,0.022458,1.000000,0.024778,0.005214,0.024289,0.020561,0.002521,0.028021,0.011281,0.065917,...,0.001808,0.013143,0.026469,0.018780,0.001947,0.003725,0.005459,0.015395,0.022458,1.000000
DWG Canyon,0.647396,0.024778,1.000000,0.114861,0.245174,0.664164,0.028527,0.724303,0.287801,0.716107,...,0.024369,0.084257,0.476112,0.224488,0.085616,0.274591,0.199578,0.144065,0.647396,0.024778
can see you,0.071693,0.005214,0.114861,1.000000,0.006649,0.089025,0.003663,0.111962,0.010734,0.069789,...,0.004084,0.014516,0.059639,0.148188,0.043470,0.016936,0.010560,0.009552,0.071693,0.005214
T1 Burdol,0.190528,0.024289,0.245174,0.006649,1.000000,0.069873,0.043682,0.081800,0.581745,0.105595,...,0.075412,0.118188,0.061293,0.366384,0.149108,0.497137,0.445523,0.123392,0.190528,0.024289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
사 용 수,0.161267,0.003725,0.274591,0.016936,0.497137,0.106717,0.057637,0.096442,0.569508,0.193322,...,0.091926,0.124589,0.047498,0.160757,0.125480,1.000000,0.455337,0.149123,0.161267,0.003725
카게게,0.102552,0.005459,0.199578,0.010560,0.445523,0.066264,0.041442,0.075599,0.416089,0.137581,...,0.211748,0.223153,0.118499,0.156191,0.095324,0.455337,1.000000,0.271636,0.102552,0.005459
IAKIL,0.177058,0.015395,0.144065,0.009552,0.123392,0.112703,0.035474,0.105005,0.124677,0.160190,...,0.069688,0.759511,0.174145,0.137120,0.034547,0.149123,0.271636,1.000000,0.177058,0.015395
jooI,1.000000,0.022458,0.647396,0.071693,0.190528,0.710296,0.038351,0.820694,0.172132,0.695309,...,0.025006,0.156684,0.527983,0.219524,0.040231,0.161267,0.102552,0.177058,1.000000,0.022458


### 상위 5개의 유사도를 가진 유저 가져옴
### 10개로할까?

In [174]:
df_user_user_sim['DWG Canyon'].sort_values(ascending=False)[1:6]

JUGKlNG      0.923532
단군의청춘부재      0.780219
By myse1f    0.775527
Ar1a         0.770979
gganknam     0.769793
Name: DWG Canyon, dtype: float64

### DWG Canyon에 대해서 추천을 처리해 줄 행을 뽑아옴(Series형태)

deepcopy로 뽑아와야 함
shallowcopy하면 reference연동되서 값이 변동됨.

In [112]:
recommend_user_mastery = copy.deepcopy(user_mastery.loc['DWG Canyon',:])
recommend_user_mastery

266    0.600000
103    0.005527
84     0.032101
12     0.005838
32     0.000000
         ...   
238    0.000000
115    0.000000
26     0.037176
142    0.059218
143    0.000642
Name: DWG Canyon, Length: 150, dtype: float64

### DWG Canyon플레이어가 한판이라도 플레이 한 (threshold=0) 챔피언 list

In [98]:
played = user_mastery.loc['DWG Canyon',:][user_mastery.loc['DWG Canyon',:]>0].index
played

Index(['266', '103', '84', '12', '523', '22', '432', '53', '63', '51', '164',
       '69', '42', '122', '131', '36', '245', '60', '28', '81', '9', '114',
       '105', '41', '79', '104', '120', '39', '427', '59', '24', '126', '222',
       '145', '429', '30', '55', '141', '121', '203', '7', '64', '876', '127',
       '236', '99', '54', '11', '21', '62', '82', '25', '267', '75', '76',
       '56', '20', '2', '61', '516', '78', '555', '246', '133', '33', '421',
       '107', '92', '68', '13', '113', '875', '35', '102', '14', '15', '72',
       '50', '517', '163', '91', '48', '4', '77', '6', '110', '45', '112', '8',
       '106', '498', '101', '5', '157', '83', '154', '26', '142', '143'],
      dtype='object')

### DWG Canyon플레이어가 한판이라도 플레이 하지 않은 (threshold=0) 챔피언 list

In [99]:
unplayed = user_mastery.loc['DWG Canyon',:][user_mastery.loc['DWG Canyon',:]==0].index
unplayed

Index(['32', '34', '1', '136', '268', '201', '31', '119', '3', '86', '150',
       '74', '420', '40', '202', '43', '38', '10', '85', '240', '96', '89',
       '117', '90', '57', '111', '518', '80', '497', '58', '235', '98', '27',
       '37', '16', '134', '223', '44', '17', '412', '18', '23', '29', '67',
       '161', '254', '19', '777', '350', '238', '115'],
      dtype='object')

### 한판이라도 플레이 안 한 챔피언 list의 점수를 업데이트 한다!

In [149]:
for champ in unplayed:
    temp_val=0;
    temp_sim=0;
    for top_k in df_user_user_sim['DWG Canyon'].sort_values(ascending=False)[1:6].index:
        temp_val +=df_user_user_sim['DWG Canyon'][top_k]*user_mastery.loc[top_k,champ]
        temp_sim +=df_user_user_sim['DWG Canyon'][top_k]
    recommend_user_mastery[champ]=temp_val/temp_sim

### 기존에 플레이 했던 챔피언들 포함 정렬

In [170]:
recommend_user_mastery.sort_values(ascending=False)[:50]

64     1.000000
163    0.823363
266    0.700000
76     0.418641
517    0.290961
141    0.277616
421    0.231038
875    0.219241
30     0.211460
126    0.202985
203    0.189640
164    0.186618
28     0.178248
60     0.174426
121    0.167814
13     0.157238
79     0.150174
39     0.133591
145    0.102285
5      0.094946
81     0.091461
120    0.089545
104    0.087308
113    0.077085
245    0.074621
202    0.070299
142    0.059218
53     0.056891
236    0.055612
20     0.054664
48     0.052021
43     0.051061
106    0.050858
876    0.047994
6      0.045746
246    0.044188
555    0.043478
92     0.042356
38     0.041429
9      0.040635
498    0.040293
2      0.040219
63     0.037813
26     0.037176
80     0.036883
154    0.036771
3      0.036422
134    0.035857
21     0.033718
84     0.032101
Name: DWG Canyon, dtype: float64

In [152]:
recommend_user_mastery.sort_values(ascending=False).index[:5]

Index(['64', '163', '266', '76', '517'], dtype='object')

### 기존에 플레이 하지 않았던 챔피언들만 정렬

In [166]:
recommend_user_mastery[unplayed].sort_values(ascending=False)

202    0.070299
43     0.051061
38     0.041429
80     0.036883
3      0.036422
134    0.035857
111    0.023177
58     0.022306
238    0.019183
40     0.018783
67     0.016664
412    0.015877
16     0.015707
96     0.014352
29     0.013352
31     0.012979
235    0.012876
223    0.012295
497    0.011954
10     0.011582
1      0.011079
90     0.009552
136    0.009417
98     0.009354
27     0.008385
119    0.006266
268    0.005986
254    0.005738
518    0.005721
18     0.005585
86     0.005186
240    0.005121
115    0.004740
89     0.003935
117    0.003590
57     0.003159
37     0.002883
777    0.002852
44     0.002459
161    0.002453
201    0.002402
34     0.002151
85     0.002148
150    0.002105
23     0.001498
17     0.001493
350    0.001293
32     0.001250
19     0.000886
74     0.000859
420    0.000252
Name: DWG Canyon, dtype: float64

In [143]:
recommend_user_mastery[unplayed].sort_values(ascending=False).index[:5]

Index(['202', '43', '38', '80', '3'], dtype='object')

## 모듈화 input(추천받을소환사명="name",상위n개의 챔피언=n, 유사도 상위 k 개=k,플레이안한기준=threshold, 기존챔프포함=True/False)
## output list

In [162]:
def recommend_champ_by_mastery(sohwan,n=5, k=10,threshold=0, include=True):
    global user_mastery
    global user_user_sim
    
    # 유사도 상위 k개의 유저 리스트
    top_k_sohwan = df_user_user_sim['DWG Canyon'].sort_values(ascending=False)[1:k+1].index
    
    # DWG Canyon에 대해서 추천을 처리해 줄 행을 뽑아옴(Series형태)
    recommend_user_mastery = copy.deepcopy(user_mastery.loc[sohwan,:])
    
    # DWG Canyon플레이어가 한판이라도 플레이 하지 않은 (threshold<=) 챔피언 list
    unplayed = user_mastery.loc['DWG Canyon',:][user_mastery.loc['DWG Canyon',:]<=threshold].index
    
    # 한판이라도 플레이 안 한 챔피언 list의 점수를 업데이트 한다!
    for champ in unplayed:
        temp_val=0;
        temp_sim=0;
        for top_k in top_k_sohwan:
            temp_val +=df_user_user_sim['DWG Canyon'][top_k]*user_mastery.loc[top_k,champ]
            temp_sim +=df_user_user_sim['DWG Canyon'][top_k]
        recommend_user_mastery[champ]=temp_val/temp_sim
    
    if include:
        return recommend_user_mastery.sort_values(ascending=False).index[:n]
    else:
        return recommend_user_mastery[unplayed].sort_values(ascending=False).index[:n]

In [172]:
recommend_champ_by_mastery('DWG Canyon',5,10,0.1,True)

Index(['64', '163', '266', '76', '517'], dtype='object')

In [173]:
recommend_champ_by_mastery('DWG Canyon',5,10,0.1,False)

Index(['104', '245', '2', '81', '59'], dtype='object')